In [ ]:
USE [master]
GO
 
CREATE OR ALTER PROCEDURE sp_FindRelated
(
  @SearchString NVARCHAR(128)
, @WildcardOption BIT = NULL --0: % in front; 1: % in back; NULL: % both sides
, @MS_ShippedOption BIT = NULL --0/1: filter added accordingly; NULL: not filtered
, @SearchScope BIT = NULL --0: Objects only; 1: Columns only; NULL: both
)AS
BEGIN
SET NOCOUNT ON;
DECLARE  
  @SearchPH NVARCHAR(24) = '!~SEARCH_PLACEHOLDER~!'
, @NewLine NVARCHAR(2) = NCHAR(10)+NCHAR(13)
, @FilterMod NVARCHAR(256) = ''
    , @ObjectQuery NVARCHAR(1024) = ''
, @ColumnQuery NVARCHAR(1024) = ''
 
 
SELECT @ObjectQuery = 
'SELECT        
   s.name [Schema]    
, ao.name [Object]     
, ao.type_desc [ObjectType]   
FROM sys.schemas s    
JOIN sys.all_objects ao ON ao.schema_id = s.schema_id    
WHERE ao.name LIKE ' + @SearchPH 
+ @NewLine + 'ORDER BY ao.name, ao.type_desc, s.name'
 
SELECT @ColumnQuery = 
'SELECT        
  ac.name [Column]  
,  t.name [ColType]                                                   
, ao.name [Object]
,  s.name [Schema] 
, ao.type_desc [ObjectType]  
FROM sys.schemas s    
JOIN sys.all_objects ao ON ao.schema_id = s.schema_id    
JOIN sys.all_columns ac ON ac.object_id = ao.object_id  
JOIN sys.types t ON t.user_type_id = ac.user_type_id AND t.system_type_id = ac.user_type_id   
WHERE ac.name LIKE '+ @SearchPH 
+ @NewLine + 'ORDER BY ac.name, ao.name, ao.type_desc, s.name'
 
 
SELECT @FilterMod = CASE
WHEN @WildcardOption = 0 
THEN '''%' + @SearchString + ''''
WHEN @WildcardOption = 1 
THEN '''' + @SearchString + '%'''
ELSE '''%' + @SearchString + '%'''
END
 
SELECT @FilterMod = CASE
WHEN @MS_ShippedOption = 0 
THEN @FilterMod + ' AND is_ms_shipped = 0'
WHEN @MS_ShippedOption = 1 
THEN @FilterMod + ' AND is_ms_shipped = 1'
ELSE @FilterMod
END
 
IF(@SearchScope = 0 OR @SearchScope IS NULL)
BEGIN
SELECT @ObjectQuery = REPLACE(@ObjectQuery, @SearchPH, @FilterMod)
SELECT 'Objects with name LIKE ' + @FilterMod + ':'
EXEC(@ObjectQuery)
END
 
IF(@SearchScope = 1 OR @SearchScope IS NULL)
BEGIN
SELECT @ColumnQuery = REPLACE(@ColumnQuery, @SearchPH, @FilterMod)
SELECT 'Columns with name LIKE ' + @FilterMod + ':'
EXEC(@ColumnQuery)
END
END
 
EXEC sp_ms_marksystemobject 'sp_FindRelated'
GO 